This notebook scrapes contents from [openreview.net](https://openreview.net/) and save them in a SQLite file.

## SQLitePipeline

In [1]:
import sqlite3
import json
import logging

from urllib.parse import urlunparse
from scrapy import Spider
from scrapy.crawler import CrawlerProcess

class SQLitePipeline(object):
    
    @classmethod
    def from_crawler(cls, crawler):
        dbpath = getattr(crawler, 'dbpath', None)
        return cls(dbpath)
    
    def __init__(self, dbpath=None):
        # SQLite connection
        self.dbconn = sqlite3.connect(dbpath or 'openreview.sqlite3')
        
        # initialize three tables: venue, publication and review
        c = self.dbconn.cursor()
        c.executescript("""
        DROP TABLE IF EXISTS venue;
        DROP TABLE IF EXISTS publication;
        DROP TABLE IF EXISTS review;
        CREATE TABLE venue (
            id text,
            name text            
        );
        CREATE TABLE publication (
            id text,
            venue text,
            number int,
            revisions int, 
            replycount int,
            tcdate int,
            tmdate int,
            signatures text,
            readers text,
            writers text,
            decision text,
            title text,
            authors text,
            authorids text,
            keywords text,
            category text,
            pdf text,
            url text,
            paperhash text,
            tldr text,
            abstract text
        );
        CREATE TABLE review (
            id text,
            type text,
            replyto text,
            venue text, 
            number int,
            tcdate int,
            tmdate int,
            date text,
            signatures text,
            readers text,
            writers text,
            decision text,
            rating text,
            confidence text,
            title text,
            review text,
            reply text
        );
        """)
        
    def process_item(self, item, spider):
        """Process item passed from Spider"""
        if item['_type_'] == 'venue':
            return self.save('venue',item)
        if item['_type_'] == 'review':
            return self.save('review',item)
        if item['_type_'] == 'publication':
            return self.save('publication',item)
        if item['_type_'] == 'commit':
            return self.dbconn.commit()
    
    
    def save(self, table, info):
        cur = self.dbconn.cursor()
        keys = [x for x in info.keys() if x != '_type_']
        keys_str = ', '.join(keys)
        qmarks = ', '.join(['?'] * len(keys))
        cur.execute(f"""
        INSERT INTO {table} ({keys_str})
        VALUES ({qmarks})
        """, [info[x] for x in keys])
    

## Scrape..

In [2]:
class OpenReviewSpider(Spider):
    name = "OpenReview"
    start_urls = ['https://openreview.net/venues']
    
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
        'ITEM_PIPELINES': {
            '__main__.SQLitePipeline': 300
        }
    }
    
    def __init__(self, *args, **kwargs):
        super(OpenReviewSpider, self).__init__(*args, **kwargs)
        
    def parse(self, response):
        """Default parser: the venue list"""
        for name, href in zip(response.css('h3 a::text').extract(),
                              response.css('h3 a::attr("href")').extract()):
            yield {
                '_type_': 'venue',
                'id': href.split('id=')[-1],
                'name': name
            }
            yield response.follow(href, callback=self.parse_venue)

    def parse_venue(self, response):
        """Parse submission list in a venue page"""
        for note_id in response.css('.submissions-list li::attr("data-id")').extract():
            href = f"https://openreview.net/notes?forum={note_id}"
            yield response.follow(href, callback=self.parse_submission)

    def parse_submission(self, response):
        """Parse submission detail page. Obtain the submission details
        and the reviews"""
        res = json.loads(response.text)['notes']
        
        def parse_review(item):
            venue_id = item['invitation'].split('/-/')[0]
            # if 'review' not in item['content']:
            #    print(item)
            review_type = item['invitation'].split('/-/')[-1]
            return {
                '_type_': 'review',
                'id': item['id'],
                'type': review_type,
                'venue': venue_id,  # redundancy for better performance
                'replyto': item['replyto'],
                'number': item['number'],
                'tmdate': item['tmdate'],
                'tcdate': item['tcdate'],
                'signatures': ', '.join(item['signatures']),
                'readers': ', '.join(item['readers']),
                'writers': ', '.join(item['writers']),
                'title': item['content'].get('title'),
                'rating': item['content'].get('rating'),
                'confidence': item['content'].get('confidence'),
                'decision': item['content'].get('decision') or item['content'].get('Acceptance decision'),
                'review': item['content'].get('review'),
                'reply': item['content'].get('reply'),
            }
            
        def parse_publication(item):
            venue_id = item['invitation'].split('/-/')[0]
            # if 'paperhash' not in item['content']:
            #    print(item)
            return {
                '_type_': 'publication',
                'id': item['id'],
                'revisions': item['revisions'],
                'venue': venue_id,
                'number': item['number'],
                'tmdate': item['tmdate'],
                'tcdate': item['tcdate'],
                'signatures': ', '.join(item['signatures']),
                'readers': ', '.join(item['readers']),
                'writers': ', '.join(item['writers']),
                'title': item['content']['title'],
                # decision may not be available if post was withdrawn
                'decision': item['content'].get('decision') or item['content'].get('Acceptance decision'),
                'tldr': item['content'].get('TL;DR'),
                'abstract': item['content']['abstract'],
                'pdf': item['content'].get('pdf'),
                'category': item['content'].get('submission category'),
                'url': item['content'].get('url'),
                'paperhash': item['content'].get('paperhash'),
                'authors': ', '.join(item['content']['authors']),
                'authorids': ', '.join(item['content']['authorids']),
                'keywords': ', '.join(item['content'].get('keywords', [])),
                'replycount': item['replyCount'],
            }
        
        for item in res:
            if '/Comment' in item['invitation']:
                continue  # skip comments
            if item.get('replyto') is not None:
                yield parse_review(item)
            else:
                yield parse_publication(item)
        
        # print(f'Finished {item["invitation"].split("/-/")[0]}', end='\r')
        
        # after all reviews for this publication is scraped
        # do commit
        yield {'_type_': 'commit'}
                
 # the script will block here until the crawling is finished

In [3]:
process = CrawlerProcess()
process.crawl(OpenReviewSpider)
process.start()

2017-12-22 10:34:32 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapybot)
2017-12-22 10:34:32 [scrapy.utils.log] INFO: Overridden settings: {}


## Verify and some examples

Install `ipython-sql`  first.
```
pip install ipython-sql
```

In [4]:
%load_ext sql
%sql sqlite:///openreview.sqlite3

'Connected: None@openreview.sqlite3'

In [5]:
%%sql
SELECT
    (SELECT count(*) from venue) AS n_venue,
    (SELECT count(*) from publication) AS n_publication,
    (SELECT count(*) from review) AS n_review

Done.


n_venue,n_publication,n_review
22,2057,14549


In [9]:
%%sql
SELECT * from venue limit 5

Done.


id,name
AKBC.ws/2013,AKBC 2013
cv-foundation.org/CVPR/2017/BNMW,CVPR 2017 BNMW
ECCV2016.org/BNMW,ECCV2016 BNMW
ICLR.cc/2013,ICLR 2013
ICLR.cc/2014,ICLR 2014


In [10]:
%%sql
SELECT id, venue, number, authors, SUBSTR(abstract, 1, 120) || '...' as abstract from publication limit 5

Done.


id,venue,number,authors,abstract
rySGOPjnb,NIPS.cc/2017/Workshop/MLITS,2,withdraw,withdraw...
HkW01LLA-,NIPS.cc/2017/Workshop/MLITS,12,"Deepak Mittal, Mudamala Avinash Reddy, Gitakrishnan Ramadurai, Kaushik Mitra, Balaraman Ravindran","Video image processing of traffic camera feeds is useful for counting and classifying vehicles, estimating queue length,..."
BypTNXUCW,NIPS.cc/2017/Workshop/MLITS,5,"Ramesh​ ​Sarukkai, ​ ​Shaohui​ ​Sun",We present the framework for an unified transport engine that allows for streamlining wide variety of data sources and d...
S1uHiFyyg,NIPS.cc/2016/workshop/MLITS,5,"Michael Treml, José Arjona-Medina, Thomas Unterthiner, Rupesh Durgesh, Felix Friedmann, Peter Schuberth, Andreas Mayr, Martin Heusel, Markus Hofmarcher, Michael Widrich, Bernhard Nessler, Sepp Hochreiter","Deep learning has considerably improved semantic image segmentation. However,its high accuracy is traded against larger..."
HylddmUAZ,NIPS.cc/2017/Workshop/MLITS,6,"Mustafa Mukadam, Akansel Cosgun, Alireza Nakhaei, Kikuo Fujimura","In this paper we consider the problem of autonomous lane changing for self driving cars in a multi-lane, multi-agent set..."


In [7]:
%%sql
SELECT id, replyto, number, signatures, rating, SUBSTR(review, 1, 120) || '...' as review from review limit 10

Done.


id,replyto,number,signatures,rating,review
S1zKiizNZ,Bk_wCFM7b,2,roboticsfoundation.org/RSS/2017/RCW_Workshop/-_Proceedings/Paper4/AnonReviewer4,"5: Top 15% of accepted papers, strong accept","+ Good review of compression algorithms, their application to underwater images, along with pros and cons+ Nice descrip..."
rkPB2MeEW,Bk_wCFM7b,1,roboticsfoundation.org/RSS/2017/RCW_Workshop/-_Proceedings/Paper4/AnonReviewer1,"4: Top 50% of accepted papers, clear accept","This submission clearly fits the overall theme of the workshop, and presents sonar image compression and decompression a..."
HkYB0n7Jf,rySGOPjnb,3,NIPS.cc/2017/Workshop/MLITS/Paper2/AnonReviewer1,1: Strong rejection,Strengths+ The goal is interesting. The paper attempts to address an interesting topic of machine learning research: t...
S1LkoPmyf,rySGOPjnb,2,NIPS.cc/2017/Workshop/MLITS/Paper2/AnonReviewer3,2: Marginally below acceptance threshold,Pros:+ The task solved by the work (traffic speed prediction in time) seems well-motivated+ Predictability of traffic ...
H1PzHDmyz,rySGOPjnb,1,NIPS.cc/2017/Workshop/MLITS/Paper2/AnonReviewer2,2: Marginally below acceptance threshold,Quality/Clarity:There are a few things that were confusing on my initial read.- I couldn't get a clear image of how sp...
SJCFQ_vJM,r1tLym8T-,1,NIPS.cc/2017/Workshop/Autodiff/Program_Chairs,None,None
BJdqn4byG,r1tLym8T-,2,NIPS.cc/2017/Workshop/Autodiff/Paper3/AnonReviewer2,"5: Top 15% of accepted papers, strong accept","The author provides an overview of useful, but underutilized, AD techniques. For example, the manuscript highlights: - ..."
H1N0Efbkf,r1tLym8T-,1,NIPS.cc/2017/Workshop/Autodiff/Paper3/AnonReviewer1,"4: Top 50% of accepted papers, clear accept","This is a great and clear overview of AD, its usefulness and variants: forward-mode vs. reverse-mode, source-transformat..."
Hy56Q_DJM,HJ0WtefAW,1,NIPS.cc/2017/Workshop/Autodiff/Program_Chairs,None,None
SyE4j4-yf,HJ0WtefAW,2,NIPS.cc/2017/Workshop/Autodiff/Paper6/AnonReviewer2,1: Strong rejection,This paper is off-topic for this workshop....
